In [3]:
import torch
import csv
import os
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, random_split
import pandas as pd
from dataset_reader import Traces_Dataset
from mlp_model import MLP
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import optuna
from optuna.trial import TrialState

In [39]:
DEVICE = torch.device("cpu")
BATCHSIZE = 128
target_features = 7
DIR = os.getcwd()
EPOCHS = 50

N_TRAIN_EXAMPLES = BATCHSIZE * 300
N_VALID_EXAMPLES = BATCHSIZE * 100

In [40]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 321
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features

    layers.append(nn.Linear(in_features, target_features))
    # layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [41]:
def get_dataset():
    dataset = Traces_Dataset('dataset_test.csv')
    dataset.split_dataset(0.95, 0.05, 0)
    dataset.clean_features()
    dataset.find_mean_std()
    dataset.normalize()
    print(dataset.inputs.shape)

    # initialize train, val, test set
    X_train = dataset[dataset.train_set.indices][0]
    Y_train = dataset[dataset.train_set.indices][1]

    X_val = dataset[dataset.val_set.indices][0]
    Y_val = dataset[dataset.val_set.indices][1]

    X_test = dataset[dataset.test_set.indices][0]
    Y_test = dataset[dataset.test_set.indices][1]

    # initialize dataloader 
    train_dataset = TensorDataset(X_train, Y_train)
    train_loader = DataLoader(train_dataset, batch_size=BATCHSIZE, shuffle=True)

    val_dataset = TensorDataset(X_val, Y_val)
    val_loader = DataLoader(val_dataset, batch_size=BATCHSIZE, shuffle=True)

    return train_loader, val_loader

In [42]:
def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_dataset()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (train_inputs, train_targets) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            # data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            train_inputs, train_targets = train_inputs.to(DEVICE), train_targets.to(DEVICE)

            optimizer.zero_grad()
            train_outputs = model(train_inputs)
            loss = nn.MSELoss()(train_outputs, train_targets)
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch_idx, (val_inputs, val_targets) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                # data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                val_inputs, val_targets = val_inputs.to(DEVICE), val_targets.to(DEVICE)

                val_outputs = model(val_inputs)
                # Get the index of the max log-probability.
                val_loss = nn.MSELoss()(val_outputs, val_targets)
                total_val_loss += val_loss.item()

            # Average validation loss for the epoch
        avg_val_loss = total_val_loss / len(valid_loader)

        trial.report(avg_val_loss, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return avg_val_loss

In [43]:
if __name__ == "__main__":
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=20, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-03-27 00:16:24,808] A new study created in memory with name: no-name-e7f0ea8c-a15c-4c81-a4bc-933af8106038


torch.Size([1000, 321])


[W 2024-03-27 00:16:25,151] Trial 0 failed with parameters: {'n_layers': 2, 'n_units_l0': 106, 'dropout_l0': 0.4176551516861762, 'n_units_l1': 13, 'dropout_l1': 0.26076988483133545, 'optimizer': 'SGD', 'lr': 0.004338569032270813} because of the following error: The value nan is not acceptable.
[W 2024-03-27 00:16:25,151] Trial 0 failed with value nan.


torch.Size([1000, 321])


[I 2024-03-27 00:16:25,409] Trial 1 finished with value: 3230.15283203125 and parameters: {'n_layers': 2, 'n_units_l0': 26, 'dropout_l0': 0.29452755895454896, 'n_units_l1': 33, 'dropout_l1': 0.4141746209705277, 'optimizer': 'SGD', 'lr': 1.170491290848602e-05}. Best is trial 1 with value: 3230.15283203125.


...

torch.Size([1000, 321])
torch.Size([1000, 321])
torch.Size([1000, 321])


[I 2024-03-27 00:16:29,335] Trial 16 finished with value: 85.67589569091797 and parameters: {'n_layers': 2, 'n_units_l0': 61, 'dropout_l0': 0.34647738617466867, 'n_units_l1': 101, 'dropout_l1': 0.45859135694848435, 'optimizer': 'Adam', 'lr': 0.003851702549424317}. Best is trial 12 with value: 68.23986053466797.
[I 2024-03-27 00:16:29,483] Trial 17 pruned. 
[I 2024-03-27 00:16:29,538] Trial 18 pruned. 


torch.Size([1000, 321])
torch.Size([1000, 321])
torch.Size([1000, 321])


[I 2024-03-27 00:16:29,872] Trial 19 finished with value: 90.40322875976562 and parameters: {'n_layers': 2, 'n_units_l0': 4, 'dropout_l0': 0.23788490246006797, 'n_units_l1': 110, 'dropout_l1': 0.37361011198862604, 'optimizer': 'RMSprop', 'lr': 0.00548727897266424}. Best is trial 12 with value: 68.23986053466797.


Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  8
  Number of complete trials:  11
Best trial:
  Value:  68.23986053466797
  Params: 
    n_layers: 2
    n_units_l0: 82
    dropout_l0: 0.2034032430966227
    n_units_l1: 126
    dropout_l1: 0.4948500916054156
    optimizer: Adam
    lr: 0.007689302990693475
